There are mainly two ways to implement Word2Vec <br>
A: Skip-Gram <br>
B: CBOW <br>

When to use the skip-gram model and when to use CBOW?

* According to the original paper, skip-gram works well with small datasets and can better represent rare words.
However, CBOW is found to train faster than skip-gram and can better represent frequent words.
* So the choice of skip-gram VS. CBOW depends on the kind of problem that we’re trying to solve.

In [1]:
from gensim import models

In [2]:
w2v = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [3]:
vect = w2v['healthy']

In [4]:
len(vect)

300

In [5]:
w2v.most_similar('happy')

[('glad', 0.7408890724182129),
 ('pleased', 0.6632170677185059),
 ('ecstatic', 0.6626912355422974),
 ('overjoyed', 0.6599286794662476),
 ('thrilled', 0.6514049172401428),
 ('satisfied', 0.6437949538230896),
 ('proud', 0.636042058467865),
 ('delighted', 0.627237856388092),
 ('disappointed', 0.6269949674606323),
 ('excited', 0.6247665286064148)]

In [6]:
sents = ['coronavirus is a highly infectious disease',
   'coronavirus affects older people the most',
   'older people are at high risk due to this disease']

In [7]:
sents = [sent.split() for sent in sents]
sents

[['coronavirus', 'is', 'a', 'highly', 'infectious', 'disease'],
 ['coronavirus', 'affects', 'older', 'people', 'the', 'most'],
 ['older',
  'people',
  'are',
  'at',
  'high',
  'risk',
  'due',
  'to',
  'this',
  'disease']]

In [10]:
custom_model = models.Word2Vec(sents, min_count=1,vector_size=300,workers=4)

##### 4. GloVe
Similar to Word2Vec, the intuition behind GloVe is also creating contextual word embeddings but given the great performance of Word2Vec. Why was there a need for something like GloVe? 

How does GloVe improve over Word2Vec?

* Word2Vec is a window-based method, in which the model relies on local information for generating word embeddings, which in turn is limited to the adjudged window size that we choose.
* This means that the semantics learned for a target word is only affected by its surrounding words in the original sentence, which is a somewhat inefficient use of statistics, as there’s a lot more information we can work with.
* GloVe on the other hand captures both global and local statistics in order to come up with the word embeddings.

When to use GloVe?

GloVe has been found to outperform other models on word analogy, word similarity, and Named Entity Recognition tasks, so if the nature of the problem you’re trying to solve is similar to any of these, GloVe would be a smart choice.
Since it incorporates global statistics, it can capture the semantics of rare words and performs well even on a small corpus.

In [11]:
import numpy as np

embeddings_dict = {}
with open('glove.6B.50d.txt','rb') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], 'float32')
        embeddings_dict[word] = vector

In [12]:
embeddings_dict[b'test']

array([ 0.13175 , -0.25517 , -0.067915,  0.26193 , -0.26155 ,  0.23569 ,
        0.13077 , -0.011801,  1.7659  ,  0.20781 ,  0.26198 , -0.16428 ,
       -0.84642 ,  0.020094,  0.070176,  0.39778 ,  0.15278 , -0.20213 ,
       -1.6184  , -0.54327 , -0.17856 ,  0.53894 ,  0.49868 , -0.10171 ,
        0.66265 , -1.7051  ,  0.057193, -0.32405 , -0.66835 ,  0.26654 ,
        2.842   ,  0.26844 , -0.59537 , -0.5004  ,  1.5199  ,  0.039641,
        1.6659  ,  0.99758 , -0.5597  , -0.70493 , -0.0309  , -0.28302 ,
       -0.13564 ,  0.6429  ,  0.41491 ,  1.2362  ,  0.76587 ,  0.97798 ,
        0.58507 , -0.30176 ], dtype=float32)

You might notice that this is a 50-dimensional vector. We downloaded the file glove.6B.50d.txt, which means this model has been trained on 6 Billion words to generate 50-dimensional word embeddings.

In [17]:
from scipy import spatial

In [19]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word:spatial.distance.euclidean(embeddings_dict[word],embedding))

In [20]:
find_closest_embeddings(embeddings_dict[b"health"])[:5]

[b'health', b'care', b'medical', b'welfare', b'prevention']

Another thing which we can use GloVe for is to transform our vocabulary into vectors. For that, we’ll use the Keras library.

##### 5. FastText

FastText is very similar to Word2Vec. However, there was still one thing that methods like Word2Vec and GloVe lacked.

As we ahve seen that Word2Vec and GloVe have in common — how we download a pre-trained model and perform a lookup operation to fetch the required word embeddings. Even though both of these models have been trained on billions of words, that still means our vocabulary is limited.
<br>
How does FastText improve over others?

FastText improved over other methods because of its capability of generalization to unknown words, which had been missing all along in the other methods.
<br>
How does it do that?

* Instead of using words to build word embeddings, FastText goes one level deeper, i.e. at the character level. The building blocks are letters instead of words.
* Word embeddings obtained via FastText aren’t obtained directly. They’re a combination of lower-level embeddings.
* Using characters instead of words has another advantage. Less data is needed for training, as a word becomes its own context in a way, resulting in much more information that can be extracted from a piece of text.

In [2]:
#!pip install fasttext

In [ ]:
import fasttext

Now to train a fasttext classifier model on any dataset, we need to prepare the input data in a certain format which is:

"__label__<label value><space><associated datapoint>"

In [ ]:
all_texts = train['text'].tolist()
all_labels = train['drug type'].tolist()
prep_datapoints=[]
for i in range(len(all_texts)):
    sample = '__label__'+ str(all_labels[i]) + ' '+ all_texts[i]
    prep_datapoints.append(sample) 

In [ ]:
with open('train_fasttext.txt','w') as f:
    for datapoint in prep_datapoints:
        f.write(datapoint)
        f.write('n')
    f.close()

In [ ]:
model = fasttext.train_supervised('train_fasttext.txt')